### Deteccion de vehiculo en área de interes y prediccion con etiqueta

In [2]:
import cv2
import time
from keras.applications.imagenet_utils import preprocess_input#, decode_predictions
from keras.models import load_model
import matplotlib.pyplot as plt
import numpy as np
from videos.funtions import roi_detection, vehicle_classifier_and_labeler

In [8]:


# Load car classifier
car_classifier = cv2.CascadeClassifier('../datasets/cars.xml')

# Load video file
cap = cv2.VideoCapture('videos/mer1_mod.mp4')

# Set video FPS to 20

cap.set(cv2.CAP_PROP_FPS,0.1)

#fps = cap.get(cv2.CAP_PROP_FPS)
#print("fps:",fps)
#load model
model = load_model("../nn_model_Mobilenet.h5")
names = ['bus', 'car', 'truck']
count = 1
# Loop over the frames from the video
while cap.isOpened():
    if count%2==0:
        # Read the next frame from the video
        ret, frame = cap.read()
        if not ret:
            break

        # Detect cars in the frame
        # Define las coordenadas del ROI (Region of interest)
        x1, y1 = 100, 550
        x2, y2 = 720, 750

        #Region of interest
        roi = frame[y1:y2,x1:x2]

        cars = roi_detection(frame,car_classifier, roi, x1,x2,y1,y2)
 
        # Para cada deteccion realizar la clasificacion y dibujar la etiqueta
        for (x,y,w,h) in cars:
            
            preds = vehicle_classifier_and_labeler(x, y, x1, y1, w, h, model, frame, roi)
            posicion_etiqueta = (x+x1 , y+y1 - 10)
            #Colocar etiqueta
            cv2.putText(frame, names[np.argmax(preds)], posicion_etiqueta, cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)
        

        # Display the resulting frame
        cv2.imshow('frame', frame)
            
    # Wait for user input to move to the next frame
    if cv2.waitKey(1) == ord('q'):
        break
    count += 1
# Release the video capture object and close all windows
cap.release()
cv2.destroyAllWindows()

1/1 [==============================] - 0s 74ms/step
